In [1]:
import random
import classes as cls
import routing
import time
from framework import Framework
import cProfile


In [2]:

def test_all_macros_connected(macros, nets):
    for macro in macros:
        connected = any(macro in net.macros for net in nets)
        assert connected, f"Macro {macro.name} is not connected to any net"



# Floorplan Setup

In [13]:
MACRO_CNT = 40
WIDTH_MAX = 150
WIDTH_MIN = 50
HEIGHT_MAX = 150
HEIGHT_MIN = 50

# NET_CNT = 20
RAND_NET_CNT = 1 # Probability of a macro being connected to a net

random.seed(42)  # For reproducibility of random numbers

macros = []
for i in range(MACRO_CNT):
    width = random.randint(WIDTH_MIN, WIDTH_MAX)
    height = random.randint(HEIGHT_MIN, HEIGHT_MAX)
    macro_id = i
    macro_name = f"m{macro_id+1}"
    macro = cls.Macro(macro_name, macro_id, width, height, [])
    pin = cls.Pin(0, 0)
    macro.pins.append(pin)
    macros.append(macro)

# Generate nets connecting the macros
net_set = set()
net_avail = set(macros)
net_cnt = {}
nets = []
LIMIT_NET_CNT = 3
for i in range(len(macros)):
    if (random.random() < RAND_NET_CNT and macros[i] in net_set and macros[i] not in net_avail): continue
    if (len(nets) == 40): break
    macro1 = macros[i]
    while True:
        macro2 = random.choice(list(net_avail))
        if macro2 != macro1:
            break
    net_set.add(macro1)
    net_set.add(macro2)
    net_name = f"n{len(nets)+1}"
    if macro1.id not in net_cnt.keys(): net_cnt[macro1.id] = 1 
    else: net_cnt[macro1.id] += 1
    if macro2.id not in net_cnt.keys(): net_cnt[macro2.id] = 1 
    else: net_cnt[macro2.id] += 1
    if net_cnt[macro1.id] == LIMIT_NET_CNT: net_avail.remove(macro1)
    if net_cnt[macro2.id] == LIMIT_NET_CNT: net_avail.remove(macro2)
    net = cls.Net(net_name, [macro1, macro2])
    nets.append(net)

# Ensure all macros are connected to at least one net
# test_all_macros_connected(macros, nets)
print("Number of macros: ", len(macros))
print("Number of nets: ", len(nets))


Number of macros:  40
Number of nets:  40


# Placement

In [14]:

FL = cls.Floor(1600,1600, 1)
FW = Framework(macros, nets, FL)
start_time=time.time()
print("___________Placement__________")
FW.place(500, genVid=0, filename=f"images/10MACROS_{len(macros)}_{len(nets)}_testhighOL_{FW.floor.h}*{FW.floor.w}.png")
end_time=time.time()
print("Placement Time: ", end_time-start_time)
start_time = time.time()
macros = FW.macros
for macro in macros:
    print(f"{macro.name} : ({macro.x}, {macro.y})")

for net in nets:
    print(f"{net.name} : {[(macro.name, macro.x, macro.y) for macro in net.macros]}")


___________Placement__________
Iteration 0 ----> l:-57456914.85281205 h:42012.23526335414  o:8971.078533991984
Iteration 10 ----> l:-29675674.62785976 h:42721.39964241951  o:4630.148941908959
Iteration 20 ----> l:-6550830.0961120585 h:41900.61116837709  o:1017.0202320224503
Iteration 30 ----> l:-28064939.44465711 h:44845.974202138386  o:4378.1396047585895
Iteration 40 ----> l:-42645705.08069826 h:39617.23787542559  o:6657.201225441067
Iteration 50 ----> l:-12272205.311852148 h:43342.0466373584  o:1910.759885189811
Iteration 60 ----> l:-14520654.665419534 h:38951.87345022581  o:2262.7660612452046
Iteration 70 ----> l:-699947.4940880705 h:38346.84268730105  o:103.37510178137023
Iteration 80 ----> l:-15390046.32972734 h:38819.28350237691  o:2398.6292259726506
Iteration 90 ----> l:-11550639.685157532 h:36530.53974979053  o:1799.0795539699598
Iteration 100 ----> l:-12299500.494684214 h:43730.73951298469  o:1914.9640242455043
Iteration 110 ----> l:-5549029.119248875 h:38491.26977795416  o:86

# Routing

In [ ]:


print("_________Routing_________")
FW.route(disp=True)
end_time = time.time()
print("Routing Time: ", end_time - start_time)

